<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo">
     
# Dask 

Dask is a parallelization library for Python that works on your laptop all the way to cluster-scale (ie. distributed multi-node)

Main focus on creating distributed array-like abstraction: Numpy- and Pandas-like behavior.

Stack:

- Array, bag, dataframe, delayed
- Graph spec
- Scheduler

Let's you focus on algorithms and not scheduling.

Tutorial: https://github.com/dask/dask-tutorial

See also some amazing new lectures/tutorials:

https://www.youtube.com/watch?v=5Md_sSsN51k&list=PLYx7XA2nY5Gf37zYZMw6OqGFRPjB1jCy6&index=17

and a shorter talk by Matt on Dask:

https://www.youtube.com/watch?v=PAGjm4BMKlk&list=PLYx7XA2nY5Gf37zYZMw6OqGFRPjB1jCy6&index=16

In [ ]:
!brew install graphviz ## on a mac
#!apt-get install graphviz ## on linux
!pip install graphviz ## dont do this with conda, installs a Python 2 package...

## Dask Arrays

Distributed notion of an array. `Dask.array` translates your array operations into a graph of inter-related tasks with data dependencies between them. Dask then executes this graph in parallel with multiple threads. We'll discuss more about this in the next section.

Manipulate `dask.array` object as you would a numpy array

In [2]:
import dask.array as da
x = da.linspace(1,10,1000000,chunks=(1000,))

In [3]:
import numpy as np
y = np.linspace(1,10,1000000)
sum(y)

5500000.0

In [4]:
x.shape

(1000000,)

In [5]:
rez = x.sum()

In [6]:
rez.compute()

5500000.0000000177

In [ ]:
import numpy as np
import dask.array as da

x = da.random.normal(10, 0.1, size=(20000, 20000),   # 400 million element array 
                              chunks=(1000, 1000))   # Cut into 1000x1000 sized chunks
y = x.mean(axis=0)[::100]                            # Perform NumPy-style operations

In [ ]:
x.nbytes / 1e9  # Gigabytes of the input processed lazily

In [ ]:
%%time
y.compute()     # Time to compute the result

In [ ]:
import numpy as np

In [ ]:
%%time 
x = np.random.normal(10, 0.1, size=(20000, 20000)) 
y = x.mean(axis=0)[::100] 
y

In [ ]:
del x

In [ ]:
%%time
x = da.random.normal(10, 0.1, size=(20000, 20000), chunks=(1000, 1000))
y = x.mean(axis=0)[::100] 
y.compute()

## Dask Dataframes

meant to mimick most of pandas dataframes, but now these dataframes can be out of core.

In [ ]:
!ls -lah ../02_Plotting_and_Viz/data/uber-raw-data-apr14.csv

In [10]:
import dask.dataframe as dd
import pandas as pd

In [11]:
%%time
df = pd.read_csv("../02_Plotting_and_Viz/data/uber-raw-data-apr14.csv")

CPU times: user 472 ms, sys: 57.3 ms, total: 530 ms
Wall time: 583 ms


In [12]:
del df

In [13]:
%%time
df = dd.read_csv("../02_Plotting_and_Viz/data/uber-raw-data-apr14.csv")

CPU times: user 8.56 ms, sys: 1.97 ms, total: 10.5 ms
Wall time: 9.48 ms


Some of the reading in is delayed, but we can still inspect the data.

In [14]:
df.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


Other operations are delayed until you compute them:

In [15]:
df.describe()

dd.DataFrame<describ..., npartitions=1>

In [16]:
df.describe().compute()

,Lat,Lon
count,564516.000000,564516.000000
mean,40.740005,-73.976817
std,0.036083,0.050426
min,40.072900,-74.773300
25%,40.722500,-73.997700
50%,40.742500,-73.984800
75%,40.760700,-73.970000
max,42.116600,-72.066600


In [17]:
df.describe()['Lat'].compute()

count    564516.000000
mean         40.740005
std           0.036083
min          40.072900
25%          40.722500
50%          40.742500
75%          40.760700
max          42.116600
Name: Lat, dtype: float64

Let's use distributed dataframes to analyze NYC Taxi data stored as CSV files on S3.
This data is stored as large CSV files on S3 in a public bucket.

(https://github.com/mrocklin/scipy-2016-parallel/blob/master/notebooks/08-distributed-dataframes.ipynb)

In [19]:
from s3fs import S3FileSystem
s3 = S3FileSystem(anon=True)

s3.ls('dask-data/nyc-taxi/2015')

['dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-02.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-03.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-04.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-05.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-06.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-07.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-08.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-09.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-10.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-11.csv',
 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-12.csv']

We would like to load this data with Pandas, but there is too much data here to fit in memory.

In [20]:
s3.info('dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv')

{'ETag': '"9ee7b2aa563752b3cc7edce719ae737e-30"',
 'Key': 'dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
 'LastModified': datetime.datetime(2016, 2, 14, 20, 7, 21, tzinfo=tzutc()),
 'Size': 1985964692,
 'StorageClass': 'STANDARD'}

In [21]:
import pandas as pd

with s3.open('dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv') as f:
    df = pd.read_csv(f, nrows=5)  # look at just five rows
    
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0,0.3,16.30


In [22]:
from dask.distributed import Executor, progress

e = Executor(set_as_default=True)
e

<Client: scheduler="127.0.0.1:8786" processes=4 cores=4>

In [23]:
import dask.dataframe as dd

df = dd.read_csv('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
                 storage_options={'anon': True})
df

dd.DataFrame<from-de..., npartitions=32>

In [24]:
df = e.persist(df)
progress(df)

Existing Pandas experience transfers over decently well to Dask.dataframe. However there are a few caveats when dealing with distributed systems:
   - Until you call e.persist (for large results) or e.compute (for small results), all computations are lazy
   - Call progress on a dataframe after you persist to track the progress of a computation. You can continue doing work immediately. All work happens in the background.
   - If you are computing a small result, just add .compute() to the end of your result, like df.passenger_count.sum().compute(). This will block and return the result when finished.

In [ ]:
positive_fares = df[df.fare_amount > 0]
fares = df[['fare_amount', 'tip_amount', 'payment_type']]

fares = e.persist(fares)  # triggers computation
progress(fares)

In [ ]:
fares.head()

In [ ]:
(fares.tip_amount == 0).sum().compute()

In [ ]:
fares.count().compute()

In [ ]:
df.passenger_count.sum().compute()

Instead, we connect to the cluster and use dask.dataframe to load the CSV data into ~700 Pandas dataframes spread across our cluster. We get back a Dask.dataframe to coordinate these small Pandas dataframes.

`dask.delayed` (a la joblib):
 
   - `delayed(function)(*args, **kwargs)` -> lazy function that hasn't yet been evaluated
   - `delayed(data)` -> lazy object that pretends to be your data
 
 See the excellent talk at SciPy 2016: https://www.youtube.com/watch?v=PAGjm4BMKlk&list=PLYx7XA2nY5Gf37zYZMw6OqGFRPjB1jCy6

In [25]:
# get a local Executor
from distributed import Executor
Executor(set_as_default=True)

<Client: scheduler="127.0.0.1:64840" processes=4 cores=4>

In [26]:
import random
from dask import delayed, visualize
from time import sleep

@delayed(pure=True)
def add(a,b):
    sleep(random.random())
    return a+b

@delayed(pure=True)
def mul(a,b):
    sleep(random.random())
    return a*b

@delayed(pure=True)
def inc(a):
    sleep(random.random())
    return a + 1

In [ ]:
x = add(1,2)
x

In [ ]:
x.compute()

In [ ]:
a = inc(1)
b = mul(1,2)
c = add(a,b)
c

In [ ]:
c.visualize(rankdir="LR")

In [ ]:
c.compute()

### Loops

In [ ]:
results = []
for x in range(4):
    a = inc(x)
    b = mul(1,x)
    c = add(a,b) - add(a,b)
    results.append(c)

total = delayed(sum,pure=True)(results)
total

In [ ]:
total.visualize(rankdir="LR")

`pure=True`: finds nested shared expressions deep in code that dont need to be recomputed. Eg. `inc(1)` here is the same so it only gets called once. A pure function should have no side-effects.

In [ ]:
total.compute()

In [ ]:
results = []
for y in range(2,10,2):
    for x in range(4):
        a = inc(1)
        b = mul(y,x)
        c = add(a,b)
        results.append(c)

total = delayed(sum,pure=True)(results)
total

In [ ]:
total.visualize()

In [ ]:
total.compute()

In [ ]:
# Tree reduction --- add up pairwise
while len(results) > 1:
    new_results = []
    
    for i in range(0,len(results),2):
        res = add(results[i], results[i+1])
        new_results.append(res)
    
    results = new_results

total = results[0]
total.visualize()

In [ ]:
total.compute()

Note: you cannot iterate over a delayed object and you can't use them in case statements (because we dont know how long they are until they've been computed)

In [ ]:
for x in range(inc(1)):
    print("hey!")

## scheduling the execution

where you run a certain piece of a parallel task depends on your architecture, what needs each piece has, and what the bottlenecks are in moving data between pieces.

The **single machine scheduler** is optimizes for larger-than-memory use. It uses:
  
   - Parallel CPU
   - Minimizes RAM: tries to remove intermediary tasks that aren't needed anymore
   - low overhead: 100$\mu$s per task
 

**Distributed scheduler** - tries to minimize data movement so you dont have to move data between computers unnecessarily.
 
 - distributed to schedule across many workers
 - works well with distributed datastores (HDFS)
 - asynchronous
 - data local
 
run `dask-scheduler` on the command line and then 

In [ ]:
from dask.distributed import Executor, progress
e = Executor(set_as_default=True)
e

# swap out concurrent.futures with a dask executor.

In [ ]:
%%time 
from time import sleep

#from concurrent.futures import ProcessPoolExecutor
#e = ProcessPoolExecutor() 

def slowfunc(x,y,delay=1):
    sleep(delay)
    return(x+y)

In [ ]:
%%time
futures = [e.submit(slowfunc,1,2, delay=1) for _ in range(100)]
[f.result() for f in futures]

In [ ]:
e.shutdown()

There are loads of ways to do mapping now in Python, [this notebook](https://github.com/mrocklin/scipy-2016-parallel/blob/master/notebooks/map-rosetta-stone.ipynb) is the Rosetta stone.

In [ ]:
da.from_array(np.array([12,3,5,5]),chunks=(3,))